<a href="https://www.kaggle.com/code/hmahida/usarrests-pca-k-means-and-agglomerative-clustering?scriptVersionId=118595052" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ["OMP_NUM_THREADS"] = '1' # To avoid KMeans memory leak on Windows with MKL userwarning

import seaborn as sns

#### This data set contains statistics, in arrests per 100,000 residents for assault, murder, and rape in each of the 50 US states in 1973. Also given is the percent of the population living in urban areas.

In [ ]:
# load data
df =  pd.read_csv('/kaggle/input/d/hmahida/usarrests/UsArrests.csv', index_col=0)
df.head()

In [ ]:
df.info(), df.describe()

#### From the dataset summary above the following observations can be made
- The dataset contains 50 rows and 5 columns.
- The four variables have vastly different means
- The variables also have vastly different variances 
- UrbanPop variable measures the percentage of the population in each state living in an urban area, which is not a comparable number to the number of murders, rapes or assaults in each state per 100,000 individuals.
- There are no null values to report in the dataset


In [ ]:
# Plot histograms
df.hist(grid=False, figsize=(10, 5))
plt.tight_layout()
plt.show()

### Crime Rate & Urban Population

In [ ]:
fig, ax = plt.subplots(figsize=(10,30))
y = np.arange(len(df.axes[0]))  # the label locations
bar_height = 0.4  # the height of the bars

# set the position of the bars on the y-axis
bar_positions = y - bar_height

assult = ax.barh(bar_positions, df.Assault, bar_height, color = 'g')
rape = ax.barh(bar_positions, df.Rape, bar_height, color = 'b', left=df.Assault)
murder = ax.barh(bar_positions, df.Murder, bar_height, color = 'r', left=df.Assault+df.Rape)
urbanpop = ax.barh(bar_positions + bar_height, df.UrbanPop, bar_height, color = 'cyan')

ax.set_yticks(y)  # set the y-ticks to be at the same position as the bars
ax.set_yticklabels(df.axes[0])  # set the y-tick labels to be the labels of the dataframe

plt.legend(['Assault','Rape','Murder','UrbanPop'])
plt.margins(y=0)
plt.show()

#### From the bar chart above the following observations can be made
- Highest Assualt Rate : Florida and North California.
- Lowest Assualt Rate : Hawaii, North Dakota, Vermont , New Hampshire and Wisconsin.

- Highest Rape Rate : Nevada and Alaska.
- Lowest Rape Rate : Maine, North Dakota,Vermont,Connecticut,New Hampshire, Wisconsin,Rhode Island and West Virginia

- Highest Murder Rate : Georgia and Missisippi
- Lowest Murder Rate : Idaho , Iowa, Maine, New Hampshire, North Dakota, Vermont and Wisconsin.

- Highest UrbanPop Rate : Nevada and Alaska.
- Lowest UrbanPop Rate : Maine, North Dakota,Vermont,Connecticut,New Hampshire, Wisconsin,Rhode Island and West Virginia


### Correlation Analysis

In [ ]:
states = df.index
corr_df = df.corr()
labels = corr_df.columns

mask_ut=np.triu(np.ones(corr_df.shape)).astype(bool)
sns.heatmap(corr_df, mask=mask_ut, annot=True, cmap="coolwarm")

#### From the heat map above the following observations can be made
- Rate of arrests for assault has very strong positive correlation with the rate of arrests for murder.
- Rate of arrests for assault has strong positive correlation with the rate of arrests for rape.
- Rate of arrests for murder has moderate positive correlation with the rate of arrests for rape.
- Urbanpopulation percentage has moderate positive correlation with the rate of arrests for rape.
- Urbanpopulation percentage has weak positive correlation with the rate of arrests for assault.
- Urbanpopulation percentage has almost no correlation with the rate of arrests for murder.

In [ ]:
sns.pairplot(df, kind='reg')

#### From the pairplots above the following observations can be made
- Murder , Assault & Rape are highly co-related to each other. 
- UrbanPop is not in co-relation with other variables. 
- Because of the highly co-related input data, PCA can be applied to reduce the number of features.

## PCA

In [ ]:
from sklearn.decomposition import PCA
np.set_printoptions(precision=2)

X = df.values.squeeze()

pca = PCA()

X_trans = pca.fit_transform(X)

df_pca = pd.DataFrame(X_trans)
df_pca.head()

In [ ]:
std = df_pca.describe().transpose()["std"]
print(f"Standard deviation: {std.values}")
print(f"Proportion of Variance Explained: {pca.explained_variance_ratio_}")
print(f"Cumulative Proportion: {np.cumsum(pca.explained_variance_)}")

In [ ]:
def biplot(score,coeff,labels=None,points=None):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())

    fig, ax = plt.subplots(figsize=(15, 15))    

    ax.scatter(xs * scalex,ys * scaley,s=5)

    for i in range(0,len(xs)):
        txt = states[i]
        ax.annotate(txt, (xs[i]* scalex, ys[i]* scaley))

    for i in range(n):
        ax.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        if labels is None:
            ax.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'green', ha = 'center', va = 'center')
        else:
            ax.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
 
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.grid()

In [ ]:
biplot(X_trans[:,0:2],np.transpose(pca.components_[0:2, :]),list(labels),list(states))
plt.show()

#### Each point on a biplot is the projected observation, transformed from the original data. The importance of each feature is indicated by the length of the arrows on the biplot. This corresponds to the magnitude of the values in the eigenvectors. From this biplot, we see that Assault and UrbanPop are the most important features as the arrows to each of these dominate the biplot.

#### This information can also be quantified as follows:

In [ ]:
# Feature importance
pd.set_option('display.float_format', lambda x: '%.3f' % x) #change precision to see more decimal places

pc1 = abs( pca.components_[0] ) #components x features - access at index 0 to get the first component
pc2 = abs( pca.components_[1] )

feat_df = pd.DataFrame()
feat_df["Features"] = list(labels)
feat_df["PC1 Importance"] = pc1
feat_df["PC2 Importance"] = pc2
feat_df

#### From the table above, it seems that the Assault has by far the highest importance in the first principle component, while UrbanPop has the highest important in the second component. These observations agree with those deduced from the biplot. From the summarised statistics from earlier on, the mean values for these features are much larger numbers than those for the other features. It is possible that because of this, these features 'swamp' the others, which results in them being ignored.

#### Standardised data

Standardise the data so that some features do not swamp the others.

In [ ]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

std_pca = PCA()
X_std_trans = std_pca.fit_transform(X_std)

df_std_pca = pd.DataFrame(X_std_trans)
df_std_pca.head()

In [ ]:
biplot(X_std_trans[:,0:2],np.transpose(std_pca.components_[0:2, :]),list(labels))
plt.show()

#### In the standardised data results abive, far more variables are being utilised to explain the variance, as the large numbers no longer dominate it. 

In [ ]:
# Feature importance

pc1 = abs( std_pca.components_[0] ) #components x features - access at index 0 to get the first component
pc2 = abs( std_pca.components_[1] )

feat_df = pd.DataFrame()
feat_df["Features"] = list(labels)
feat_df["PC1 Importance"] = pc1
feat_df["PC2 Importance"] = pc2
feat_df

#### Inspecting the feature importance now, it seems that most of the variables contribute fairly evenly, with only some with low importance.

In [ ]:
# Cumulative variance plot
plt.ylabel('Explained variance')
plt.xlabel('Components')
plt.plot(range(1,len(std_pca.explained_variance_ratio_ )+1),
         np.cumsum(std_pca.explained_variance_ratio_),
         c='red')
plt.title("Cumulative Explained Variance")

In [ ]:
# Scree plot
plt.plot(std_pca.explained_variance_ratio_)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.title("Scree plot")
plt.show()

#### From the plots above, it seems the first 3 principal components together explain around 95% of the variance. We can therefore use them to perform cluster analysis. 

In [ ]:
pca_df = pd.DataFrame(X_std_trans[:,0:3], index = df.index)
pca_df.head()

## K-Means

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 3)

### Determination of the Optimal Number of Clusters

In [ ]:
def eval_Kmeans(x, k, r):
    kmeans = KMeans(n_clusters=k, random_state=r)
    kmeans.fit(x)
    return kmeans.inertia_


def elbow_Kmeans(x, max_k=10, r=123):
    within_cluster_vars = [eval_Kmeans(x, k, r) for k in range(1, max_k+1)]
    plt.plot(range(1, 11), within_cluster_vars, marker='o')
    plt.xlabel('K')
    plt.ylabel('Inertia')
    plt.show()

x_1 = pca_df[[0,1]]
X = pca_df[[0,1]].values.reshape(-1, 2)
elbow_Kmeans(X)

#### The elbow runs from k=2 to k=4. In cases like this, it is not clear which value within the elbow is the most optimal, so we can investigate likely candidates further using the silhouette score.

In [ ]:
from sklearn.metrics import silhouette_score

def scatter_Kmeans(x, k, r=123):
    ''' This function takes dataframe (x),k and random_state(r) paremeters to build K-Nearest Neighbours model, 
    calculate accuracy of the model and create a scatter plot showing the clusters predicted by the model. 
    '''
    X = x.values.reshape(-1,2)
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=r)
    y_pred = kmeans.fit_predict(X)
    colours = 'rbgcmy'
    
    for c in range(k):
      plt.scatter(X[y_pred == c, 0], X[y_pred == c,1], c = colours[c], label = 'Cluster {}'.format(c))
      plt.scatter(kmeans.cluster_centers_[c, 0], kmeans.cluster_centers_[c, 1], marker='x', c = 'black')

    score = round(silhouette_score(X, kmeans.labels_, metric='euclidean'),2)
    plt.title('silhouette={}'.format(score), loc='right', fontdict={'fontsize': 16}, pad=-14)
    plt.xlabel(x.axes[1][0])
    plt.ylabel(x.axes[1][1])
    plt.legend()
    plt.show()

for k in range(2,5):
  scatter_Kmeans(x_1, k, r=0)

#### Thus we see that the best silhouette score is actually achieved using k=2.

In [ ]:
from sklearn.cluster import KMeans

# We extract the first two components
x = X_std_trans[:,0]
y = X_std_trans[:,1]

# Fit k-means
k=2
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
cluster_labels = kmeans.fit_predict(pca_df)
cent = kmeans.cluster_centers_

# Plot clusters
fig, ax = plt.subplots(figsize=(10,10))
colours = 'rbgy'
for i in range(0,k):
    ax.scatter(x[cluster_labels == i],y[cluster_labels == i],c = colours[i])
    ax.scatter(kmeans.cluster_centers_[i, 0], kmeans.cluster_centers_[i, 1], marker='o', c = "black", s=300, alpha=0.5)

for i in range(0,len(x)):
        txt = states[i]
        ax.annotate(txt, (x[i], y[i]))
ax.set_title("K-Means cluster plot")
ax.set_xlabel("Dim 2")
ax.set_ylabel("Dim 1")

In [ ]:
# Boxplots to display distribution of crime rates for the states in cluster 0 and cluster 1.
df['cluster labels']=cluster_labels
fig, ax = plt.subplots(nrows=1,ncols=3, figsize=(15,5))
sns.boxplot(x='cluster labels', y='Murder', data=df, ax=ax[0])
sns.boxplot(x='cluster labels', y='Rape', data=df, ax=ax[1])
sns.boxplot(x='cluster labels', y='Assault', data=df, ax=ax[2])

#### From the box plots above, it seems
- The states in Group-0 seems to be Low-risk States where there are relativley less Murders,Assaults and Rapes.
- The states in the Group-1 seems to have higher crime rates and can be regarded as High-risk States.

In [ ]:
for i in range(k):
    group_indices = np.argwhere(cluster_labels==i).transpose()[0]
    group = np.array(states)[group_indices]
    print(f'Group {i} States:', *group, sep=", ")
    print("\n")     

## Agglomerative clustering
As with this dataset, the goal of the analysis is to identify the unknown pattern of the data without any prior assumption, agglomerative clustering is a better choice.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

X = pca_df.values.reshape(-1, 3)
linkage_types = ['single', 'complete', 'average', 'ward']

plt.figure(figsize=[15,3])
for i, l in enumerate(linkage_types):
  plt.subplot(1, 4, i+1)
  cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage=l)
  cluster.fit_predict(X)

  ax = plt.scatter(X[:,0], X[:,1], c=cluster.labels_)
  plt.title('Clusters based on {} linkage'.format(l))
 
L = 'complete'

In [ ]:
# Visualisation
from scipy.cluster.hierarchy import dendrogram, linkage

plt.figure(figsize=(20,5))
plt.title("Single linkage based Euclidean metric Dendogram")
dend = dendrogram(linkage(X, method='single', metric='euclidean'), labels=pca_df.index)

In [ ]:
# Print dendogram using complete linkage and euclidean matric
plt.figure(figsize=(20,5))
plt.title("Complete linkage based Euclidean metric Dendogram")
dend = dendrogram(linkage(X, method='complete', metric='euclidean'), labels=pca_df.index)

In [ ]:
# Print dendogram using average linkage and euclidean matric
plt.figure(figsize=(20,5))
plt.title("Average linkage based Euclidean metric Dendogram")
dend = dendrogram(linkage(X, method='average', metric='euclidean'), labels=pca_df.index)

In [ ]:
# With Ward method
plt.figure(figsize=(20,5))
plt.title('Ward linkage based Euclidean metric Dendogram')
dend = dendrogram(linkage(X, method  = "ward"), labels=pca_df.index)

#### From the dendrogram above, it seems there are two clusters.

In [ ]:
# We extract the first two components
x = X_std_trans[:,0]
y = X_std_trans[:,1]

# Run agglomerative hierarchical clustering with 2 number of clusters, 
#'complete' linkage method and 'euclidean' distance metric used for that dendrogram).
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='complete')
cluster_labels = cluster.fit_predict(pca_df)

# Plot clusters
fig, ax = plt.subplots(figsize=(10,10))
colours = 'rbgy'
for i in range(0,k):
    ax.scatter(x[cluster_labels == i],y[cluster_labels == i],c = colours[i]) 

for i in range(0,len(x)):
        txt = states[i]
        ax.annotate(txt, (x[i], y[i]))
ax.set_title("Agglomerative cluster plot")
ax.set_xlabel("Dim 2")
ax.set_ylabel("Dim 1")

In [ ]:
# Verify the clusters obtained by using the Silhouette score 
kmeans = KMeans(n_clusters=2, init='k-means++', random_state=1)
y_pred = kmeans.fit_predict(X)
score = round(silhouette_score(X, kmeans.labels_, metric='euclidean'), 2)
score

In [ ]:
# Boxplots to display distribution of crime rates for the states in cluster 0 and cluster 1.
df['cluster labels']=cluster_labels
fig, ax = plt.subplots(nrows=1,ncols=3, figsize=(15,5))
sns.boxplot(x='cluster labels', y='Murder', data=df, ax=ax[0])
sns.boxplot(x='cluster labels', y='Rape', data=df, ax=ax[1])
sns.boxplot(x='cluster labels', y='Assault', data=df, ax=ax[2])

#### From the box plots above, it seems
- The states in the Group-0 seems to have higher crime rates and can be regarded as High-risk States.
- The states in Group-1 seems to be Low-risk States where there are relativley less Murders,Assaults and Rapes.

In [ ]:
for i in range(k):
    group_indices = np.argwhere(cluster_labels==i).transpose()[0]
    group = np.array(states)[group_indices]
    print(f'Group {i} States:', *group, sep=", ")
    print("\n")